## MANIPULACIÓN DE DATOS (SPARK SQL EN DATABRICKS)

In [0]:
from pyspark.sql import SparkSession # Puerta de entrada para trabajar con spark <-- SIEMPRE DEBEMOS IMPORTAR LA LLAVE MAESTRA QUE INICIA TODO.
from pyspark.sql.functions import *  # Funciones propias del módulo SQL de Spark, para trabajar sobre Dataframes.

spark = SparkSession.builder.appName("DatabricksSparkSQL").getOrCreate()

### DATAFRAMES (Versión Databricks Free Edition)

#### Link de [Databrick Free Edition](https://dbc-89f542f8-2df6.cloud.databricks.com/?o=758509963140561)

Para abordar el tema de **Manipulación de Datos**, utilizaremos una de las principales herramientas que ofrece **Databricks: Spark SQL**.  
Esta herramienta nos permite aplicar conceptos del lenguaje SQL sobre conjuntos de datos estructurados, ya sea que provengan directamente de archivos o estén gobernados a través del servicio **Unity Catalog**.

Gracias a Spark SQL, podremos consultar, transformar y analizar datos de forma eficiente, aprovechando tanto la potencia del motor distribuido de Spark como la organización lógica que brinda Unity Catalog.


##### FASE 1. FUENTES DE DATOS

In [0]:
#### ➡️ Como se venía explicando, Unity Catalog trabaja bajo una jerarquía de Catálagos, Esquemas, Volumenes y Tablas.
####     Por ende, es diferente la manera de acceder a su contenido.

#====== 📌 LEER ARCHIVO CSV 
##📝 SINTAXIS:
#   dataset_nombre = spark.read.option("header","true").option("inferSchema","true").csv("/Volumes/NombreCatalago/NombreEsquema/NombreArchivoCSV")

#---> spark.read : Permite leer un archivo basado en una ruta. Siempre instanciando primero SparkSession.
#---> .option("header","true||false") : Permite establecer si tomará en cuenta los encabezados del dataset. true(mostrar) | false(ocultar)
#---> .option("inferSchema","true||false") : Permite reconocer el tipo de dato de cada columna del dataset. true(mostrar) | false(ocultar)
#---> .csv(RutaArchivoCSV) : Tipo de archivo a leer (En este caso un CSV).

## 📝 EJEMPLO:
df_titanic_csv = spark.read.option("header","true").option("inferSchema","true").csv("/Volumes/explicacion_unity_catalog/ejemplos/csv_titanic")
# df_titanic_csv.show(5)

#====== 📌 LEER ARCHIVO PARQUET
##📝 SINTAXIS:
#   dataset_nombre = spark.read.parquet("/Volumes/NombreCatalago/NombreEsquema/NombreArchivoPARQUET")

#---> spark.read : Permite leer un archivo basado en una ruta. Siempre instanciando primero SparkSession.
#---> .parquet(RutaArchivoPARQUET) : Tipo de archivo a leer (En este caso un PARQUET).

## 📝 EJEMPLO
df_titanic_parquet = spark.read.parquet("/Volumes/explicacion_unity_catalog/ejemplos/parquet_titanic")
# df_titanic_parquet.show(5)

#====== 📌 LEER ARCHIVO DELTA
## 📝 SINTAXIS:
#   dataset_nombre = spark.read.format("delta").load("/Volumes/NombreCatalago/NombreEsquema/NombreArchivoDELTA")

#---> spark.read : Permite leer un archivo basado en una ruta. Siempre instanciando primero SparkSession.
#---> .format("delta") : stablece el tipo de archivo a leer (En este caso aplica siempre para DELTA)
#---> .load(RutaArchivoDelta) : Aplica solo a los archivos DELTA.

## 📝 EJEMPLO:
df_titanic_archivo_delta = spark.read.format("delta").load("/Volumes/explicacion_unity_catalog/ejemplos/archivo_delta_titanic")
# df_titanic_archivo_delta.show(5)

#====== 📌 LEER TABLA DELTA (RECOMENDADO Y NATIVO EN DATABRICKS-UNITY-CATALOG)
## 📝 SINTAXIS:
#   dataset_nombre = spark.sql("SELECT * FROM NombreCatalago.NombreEsquema.NombreTablaDelta")

#---> spark.sql() : Permite leer mediante lenguaje SQL una tabla delta. Siempre instanciando primero SparkSession.
#---> "NombreCatalago.NombreEsquema.NombreTablaDelta" : Ruta aplicable solo para las tablas delta y gobernadas por Unity Catalog

## 📝 EJEMPLO
df_titanic_delta = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic")
# df_titanic_delta.show(5)

##### FASE 2. EXPLORACIÓN INICIAL

Utilizaremos de ejemplo el dataset de titanic ( en formato tabla delta y CSV)

In [0]:
#====== A). Quitar encabezados originales provenientes del Dataset (Archivos CSV).
df_titanic_archivo_delta = spark.read.csv("/Volumes/explicacion_unity_catalog/ejemplos/csv_titanic") ## ⬅️ Archivo CSV
df_titanic_archivo_delta.show(3)
#### 🧠💡 Importante: Cuando leamos un archivo CSV, si no establecemos .option("header","true")
####                   el dataframe mostrará las columnas con un prefijo c_ seguido de una posición asignada.
####                   (Y los nombres de columnas serán parte de los registros del dataset)

In [0]:
#====== B). Mostrar los N primeros registros de un dataset.

df_titanic_delta = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic") ## ⬅️ Tabla delta

#### 📝 SINTAXIS: dataset_nombre.show(NúmeroDeRegistros)

#### 📝 EJEMPLO 1:
# df_titanic_delta.show(10) ## ⬅️ Mostrar los 10 primeros registros.

#### 📝 EJEMPLO 2:
# df_titanic_delta.show(5) ## ⬅️ Mostrar los 5 primeros registros.

#### 📝 EJEMPLO 3:
df_titanic_delta.show(3) ## ⬅️ Mostrar los 3 primeros registros.

In [0]:
#====== C). Mostrar los N últimos registros de un dataset (Retornado en una lista).

df_titanic_delta = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic") ## ⬅️ Tabla delta

#### 📝 SINTAXIS: dataset_nombre.tail(NúmeroDeRegistros)

#### 📝 EJEMPLO 1:
# df_titanic_delta.tail(10) ## ⬅️ Mostrar los 10 primeros últimos.

#### 📝 EJEMPLO 2:
# df_titanic_delta.tail(5) ## ⬅️ Mostrar los 5 primeros últimos.

#### 📝 EJEMPLO 3:
df_titanic_delta.tail(3) ## ⬅️ Mostrar los 3 primeros últimos.

In [0]:
#====== D). Mostrar el volúmen del dataset (Cantidad de filas y columnas respectivamente)

df_titanic_delta = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic") ## ⬅️ Tabla delta

#### 📝 SINTAXIS: 
#       dataset_nombre.count() ⬅️ Cantidad de filas
#       len([i for i in dataframe_nombre.columns]) ⬅️ Cantidad de columnas

#### 📝 EJEMPLO 1:
print(f"Cantidad de filas: {df_titanic_delta.count()}") ## ⬅️ Cantidad de filas.

#### 📝 EJEMPLO 2:
print(f"Cantidad de columnas: {len([i for i in df_titanic_delta.columns])}") ## ⬅️ Cantidad de columnas.